# Seguimiento con drones
En esta practica se generara la programacion del drone aplicando conocimientos de tratamiento de imagenes para identificar hacia donde se tiene que desplazar el drone

## Ejercicios

<ol>
<li>
Realizar un progrma en Python con el cual se puedan identificar objetos y que al mover estos objetos la camara de el dron identifique hacia donde tiene que desplazarse sin hacer el desplazamiento.
</li>
</ol>

**1. Realizar un progrma en Python con el cual se puedan identificar objetos y que al mover estos objetos la camara de el dron identifique hacia donde tiene que desplazarse sin hacer el desplazamiento.**

Modifica el codigo para que en lugar de hacerlo con una camara abra la camara del drone, realiza la conexión del drone y ejecute el codigo.

Una vez que se prueba que se conecta el drone y funciona correctamente la deteccion del objeto seleccionado modificar el codigo para poder iniciar el despegue y manipular el dron con el objeto.

In [2]:
# Librerias
from djitellopy import Tello
import numpy as np
import cv2 as cv

In [3]:
# Variables
frameWidth = 640
frameHeight = 480
cap = cv.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)

deadZone = 100
startCounter = 0
global imgContour

In [4]:
# Conexión y variables para drone
global dir
me = Tello()
me.connect()
me.for_back_velocity = 0
me.left_rigth_velocity = 0
me.up_down_velocity = 0
me.yaw_velocity = 0
me.speed = 0

print(me.get_baterry())

me.streamoff()
me.streamon()


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[WARNING] tello.py - 448 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 438 - Send command: 'command'


KeyboardInterrupt: 

In [5]:
#Funcion para trackbar
def empty(a):
    pass

In [6]:
#Creacion de Trackbar
cv.namedWindow("HSV")
cv.resizeWindow("HSV",640,240)
cv.createTrackbar("HUE Min","HSV",19,300,empty)
cv.createTrackbar("HUE Max","HSV",35,300,empty)
cv.createTrackbar("SAT Min","HSV",107,255,empty)
cv.createTrackbar("SAT Max","HSV",255,255,empty)
cv.createTrackbar("VALUE Min","HSV",89,255,empty)
cv.createTrackbar("VALUE Max","HSV",255,255,empty)

cv.namedWindow("Parameters")
cv.resizeWindow("Parameters",640,240)
cv.createTrackbar("Threshold1","Parameters",166,255,empty)
cv.createTrackbar("Threshold2","Parameters",171,255,empty)
cv.createTrackbar("Area","Parameters",3750,30000,empty)

In [4]:
#Verificacion de imagenes
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv.cvtColor( imgArray[x][y], cv.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv.cvtColor(imgArray[x], cv.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [ ]:
#Busqueda de contornos
def getContours(img,imgContour):
    global dir

    contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv.contourArea(cnt)
        areaMin = cv.getTrackbarPos("Area", "Parameters")
        if area > areaMin:
            cv.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            peri = cv.arcLength(cnt, True)
            approx = cv.approxPolyDP(cnt, 0.02 * peri, True)
            print(len(approx))
            x , y , w, h = cv.boundingRect(approx)

            cx = int(x + (w / 2))
            cy = int(y + (h / 2))

            if (cx <int(frameWidth/2)-deadZone):
                cv.putText(imgContour, " GO LEFT " , (20, 50), cv.FONT_HERSHEY_COMPLEX,1,(0, 0, 255), 3)
                cv.rectangle(imgContour,(0,int(frameHeight/2-deadZone)),(int(frameWidth/2)-deadZone,int(frameHeight/2)+deadZone),(0,0,255),cv.FILLED)
                dir = 1
            elif (cx > int(frameWidth / 2) + deadZone):
                cv.putText(imgContour, " GO RIGHT ", (20, 50), cv.FONT_HERSHEY_COMPLEX,1,(0, 0, 255), 3)
                cv.rectangle(imgContour,(int(frameWidth/2+deadZone),int(frameHeight/2-deadZone)),(frameWidth,int(frameHeight/2)+deadZone),(0,0,255),cv.FILLED)
                dir = 2
            elif (cy < int(frameHeight / 2) - deadZone):
                cv.putText(imgContour, " GO UP ", (20, 50), cv.FONT_HERSHEY_COMPLEX,1,(0, 0, 255), 3)
                cv.rectangle(imgContour,(int(frameWidth/2-deadZone),0),(int(frameWidth/2+deadZone),int(frameHeight/2)-deadZone),(0,0,255),cv.FILLED)
                dir = 3
            elif (cy > int(frameHeight / 2) + deadZone):
                cv.putText(imgContour, " GO DOWN ", (20, 50), cv.FONT_HERSHEY_COMPLEX, 1,(0, 0, 255), 3)
                cv.rectangle(imgContour,(int(frameWidth/2-deadZone),int(frameHeight/2)+deadZone),(int(frameWidth/2+deadZone),frameHeight),(0,0,255),cv.FILLED)
                dir = 4
            else: 
                dir = 0

            cv.line(imgContour, (int(frameWidth/2),int(frameHeight/2)), (cx,cy),
                     (0, 0, 255), 3)
            cv.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5)

            cv.putText(imgContour, "Points: " + str(len(approx)), (x + w + 20, y + 20), cv.FONT_HERSHEY_COMPLEX, .7,
                        (0, 255, 0), 2)
            cv.putText(imgContour, "Area: " + str(int(area)), (x + w + 20, y + 45), cv.FONT_HERSHEY_COMPLEX, 0.7,
                        (0, 255, 0), 2)
            cv.putText(imgContour, " " + str(int(x))+ " "+str(int(y)), (x - 20, y- 45), cv.FONT_HERSHEY_COMPLEX, 0.7,
                        (0, 255, 0), 2)
        else:
            dir = 0

In [ ]:
#Funcion para dibujar cuadrantes
def display(img):
    cv.line(img,(int(frameWidth/2)-deadZone,0),(int(frameWidth/2)-deadZone,frameHeight),(255,255,0),3)
    cv.line(img,(int(frameWidth/2)+deadZone,0),(int(frameWidth/2)+deadZone,frameHeight),(255,255,0),3)

    cv.circle(img,(int(frameWidth/2),int(frameHeight/2)),5,(0,0,255),5)
    cv.line(img, (0,int(frameHeight / 2) - deadZone), (frameWidth,int(frameHeight / 2) - deadZone), (255, 255, 0), 3)
    cv.line(img, (0, int(frameHeight / 2) + deadZone), (frameWidth, int(frameHeight / 2) + deadZone), (255, 255, 0), 3)

In [ ]:
while True:

    # Mostrar frames drone
    frame_read = me.get_frame_read()
    myFrame = frame_read.frame
    img = cv.resize(myFrame, frameWidth, frameHeight)
    if img is None:
        break
    imgContour = img.copy()
    imgHsv = cv.cvtColor(img,cv.COLOR_BGR2HSV)

    h_min = cv.getTrackbarPos("HUE Min","HSV")
    h_max = cv.getTrackbarPos("HUE Max", "HSV")
    s_min = cv.getTrackbarPos("SAT Min", "HSV")
    s_max = cv.getTrackbarPos("SAT Max", "HSV")
    v_min = cv.getTrackbarPos("VALUE Min", "HSV")
    v_max = cv.getTrackbarPos("VALUE Max", "HSV")

    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    mask = cv.inRange(imgHsv,lower,upper)
    result = cv.bitwise_and(img,img, mask = mask)
    mask = cv.cvtColor(mask, cv.COLOR_GRAY2BGR)

    imgBlur = cv.GaussianBlur(result, (7, 7), 1)
    imgGray = cv.cvtColor(imgBlur, cv.COLOR_BGR2GRAY)
    threshold1 = cv.getTrackbarPos("Threshold1", "Parameters")
    threshold2 = cv.getTrackbarPos("Threshold2", "Parameters")
    imgCanny = cv.Canny(imgGray, threshold1, threshold2)
    kernel = np.ones((5, 5))
    imgDil = cv.dilate(imgCanny, kernel, iterations=1)
    getContours(imgDil, imgContour)
    display(imgContour)

    # Volar
    if startCounter == 0:
        me.takeoff()
        startCounter = 1

    # Direcciones
    if dir == 1:
        me.yaw_velocity = - 60
    elif dir == 2:
        me.yaw_velocity = 60
    elif dir == 3:
        me.up_down_velocity = 60
    elif dir == 4:
        me.up_down_velocity = -60
    else:
        me.left_right_velocity = 0; me_for_back_velocity = 0; me.up_down_velocity = 0; me.yaw_velocity = 0
    
    # Velocidades:
    if me.send_rc_control:
        me.send_rc_control(me.left_right_velocity, me.for_back_velocity, me.up_down_velocity, me.yaw_velocity)

    stack = stackImages(0.7,([img,result],[imgDil,imgContour]))

    cv.imshow('Horizontal Stacking', stack)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()